# Embedded RK method with adaptive stepsize control

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from embedded_rk_method import embedded_rk_step
from implicit_RK_method import implicit_rk_step

#### Model of restricted three body problem

In [ ]:
# Constants
M_M=0.012277471
M_E=1-M_M
y0=np.array([0.994,0,0,-2.001585106])
T=51.19
LAM= M_E/(M_M+M_E)
MU= M_M/(M_M+M_E)

In [ ]:
# Functions
def N_E(y,t):
    n_e=(y[0]+MU)^2+y[2]^2
    return n_e

def N_M(y,t):
    n_m=(y[0]-LAM)^2+y[2]^2
    return n_m

def F(y,t):
    f1=y[1]
    f2=y[0]+2*y[4]-LAM*(y[0]+MU)/N_E(y,t)**(3/2)-MU*(y[0]-LAM)/N_M(y,t)**(3/2)
    f3=y[3]
    f4=y[2]-2*y[1]-LAM*y[2]/N_E(y,t)**(3/2)-MU*y[2]/N_M(y,t)**(3/2)
    f=np.array([f1,f2,f3,f4])
    return f

def DyF(y,t):
    df2_dx = 1 - LAM * (N_E(y)**(3/2) - 3*(y[0]+MU)**2*N_E(y)**(1/2))/(N_E(y)**3) - MU * (N_M(y)**(3/2) - 3*(y[0]-LAM)*2*N_M(y)**(1/2))/(N_M(y)**3)
    df2_dy = 3*LAM*(y[0]+MU)*N_E(y)**(-5/2)*y[2] + 3*MU*(y[0]-LAM)*N_M(y)**(-5/2)*y[2]
    df4_dx = 3*LAM*y[2]*(y[0]+MU)*N_E(y)**(-5/2) + 3*MU*y[2]*(y[0]-LAM)*N_M(y)**(-5/2)
    df4_dy = 1 - LAM*(N_E(y)**(3/2) - 3*y[2]**2*N_E(y)**(1/2))/(N_E(y)**3) - MU*(N_M(y)**(3/2) - 3*y[2]**2*N_M(y)**(1/2))/(N_M(y)**3)
    D_yf = np.array([[0,1,0,0],[df2_dx,0,df2_dy,2],[0,0,0,1],[df4_dx,-2,df4_dy,0]])
    return D_yf

In [ ]:
# Using RK4 scheme
N=500000
h=T/N

In [ ]:
# Initialize
t = np.linspace(0, T, N+1)  
y = np.zeros((2, N+1))
k0 = np.zeros(2)    
y[:,0] = y0